# Unconstrained optimization: the Truncated Newton Method
* ## Giacomo Bacchetta, bacchetta.1840949@studenti.uniroma1.it
* ## Edoardo Cesaroni, cesaroni.1841742@studenti.uniroma1.it 
* ## Fabio Ciccarelli, ciccarelli.1835348@studenti.uniroma1.it 

## Support's functions

We import the python libraries useful for the implementation of the method. These are:
- $\bf{Autograd.numpy}$, which allows us to use simple mathematical functions;
- $\bf{Autograd}$, used to calculate the gradient (*grad*) of several functions.

In [1]:
import autograd.numpy as np
from autograd import grad

## Armijo Condition

The $\bf{Armijo}$ condition is a line search method used to determine the amount to move along a given search direction. This line search algorithm can be expressed as follows:
1. Set $ \alpha > 0,\delta \in (0,1), \gamma \in (0,\frac{1}{2})$
2. Until the condition is satisfied that $f(x_k + \alpha \cdot d) > f(x_k) + \alpha \cdot \gamma \cdot (\nabla(f(x_k)) \times  d)$, then $\alpha = \alpha \cdot \delta$

In [2]:
def armijo(f, x_k, d):
    alpha = 1
    delta = 0.5
    gamma = 1e-3

    while True:
        if f(x_k + alpha*d) > f(x_k) + alpha * gamma * (grad(f)(x_k) @ d):
            alpha = delta * alpha
        else:
            return alpha

## Other useful functions

The $\bf{mxv}$ function has the goal to return the result of the product between a matrix and a vector. In particular, we applied the incremental ratio formula to carry out this multiplication.\
The $\bf{gradfi}$ function insted has the goal to return the sum between the value of *mxv* and the value of the gradient of the objective function.

In [3]:
def mxv(v, f, x):
    eta = 1e-6
    x_succ = x + eta*v
    return (grad(f)(x_succ) - grad(f)(x))/eta


def gradfi(d,f,x):
    return mxv(d,f,x) + grad(f)(x)

# Truncated Newton Method

## Descent direction

In [4]:
def dt(f, x, k):

    epsilon_1 = 0.5
    epsilon_2 = 0.5
    p = 0
    
    s = -grad(f)(x)
    
    if (s @ mxv(s, f, x)) < (epsilon_1 * (np.linalg.norm(s))**2):
        d = -(grad(f)(x))
        return d
    
    while True:
        
        if (s @ mxv(s, f, x)) <= 1e-9:              #default = 1e-9
            return -grad(f)(x)
        
        alfa = -((gradfi(p, f, x) @ s) / (s @ mxv(s, f, x)))
        p = p + alfa * s

        if np.linalg.norm(gradfi(p, f, x)) <= (1/(k+1))*epsilon_2*(np.linalg.norm(grad(f)(x))):
            d = p
            return d
        
        else:
            beta = (gradfi(p, f, x) @ mxv(s, f, x)) / (s @ mxv(s, f, x))
            s = -(gradfi(p, f, x)) + beta * s

            if (s @ mxv(s, f, x)) < (epsilon_1 * (np.linalg.norm(s))**2):
                d = p
                return d

## Main

In [5]:
def nt(f, x, eps = 1e-5):  
    
    k = 0
    
    if np.linalg.norm(grad(f)(x)) < eps:
        return x
    
    while True:
        
        d = dt(f, x, k)
        a = armijo(f, x, d)
        x = x + a * d
        
        if np.linalg.norm(grad(f)(x)) <= eps:
            print('NT: STOP! The optimum point is:', x, 'The optimum value of the objective function is:', f(x))
            return x
        
        k = k + 1

## Implementation test

From now on, we will define unconstrained problems to which we will apply our method. The solutions can be compared with those on the attached pdf file.

### Wood Function

In [6]:
def x0_wood():
    x = np.array([-3. ,-1. ,-3. ,-1.])
    return x

def wood(x):
    return 100*(x[0]**2-x[1])**2 + (x[0]-1.)**2 + (x[2]-1.)**2 + 90.*(x[2]**2-x[3])**2 + 10.1*((x[1]-1.)**2 + (x[3]-1.)**2.) + 19.8*(x[1]-1.)*(x[3]-1.)

nt(wood, x0_wood())

NT: STOP! The optimum point is: [1.00000001 1.00000002 1.         1.        ] The optimum value of the objective function is: 2.2733433534941187e-15


array([1.00000001, 1.00000002, 1.        , 1.        ])

### Scaled Rosenbrock Function

In [7]:
def rosenbrock(x):
    return sum((100*(x[i+1]-x[i]**2)**2 + (1-x[i])**2) for i in range(len(x)-1))

def x0_rosenbrock(n):
    x_0 = np.zeros(n)
    for i in range(0, n-1, 2):
        x_0[i] = -1.2
        x_0[i+1] = 1
    return x_0

nt(rosenbrock, x0_rosenbrock(2))

NT: STOP! The optimum point is: [0.99999211 0.99998419] The optimum value of the objective function is: 6.237058410058728e-11


array([0.99999211, 0.99998419])

### Cube Function

In [8]:
def x0_cube():
    return np.array([-1.2, 1])

def cube(x):
    c = 10**2
    return c*(x[1] - x[0]**3) **2 + (1 - x[0])**2

nt(cube, x0_cube())

NT: STOP! The optimum point is: [0.99998505 0.9999551 ] The optimum value of the objective function is: 2.2380602998052338e-10


array([0.99998505, 0.9999551 ])

### Powell Function

In [9]:
def powell(x):
    n = len(x)
    return sum((x[4*i-3]+10*x[4*i-2])**2+5*(x[4*i-1]-x[4*i])**2 + (x[4*i-2]-2*x[4*i-1])**4 + 10*(x[4*i-3]-x[4*i])**4 for i in range(int(n/4)))

def x0_powell(n):
    x = np.zeros(n)
    for i in range(0, n, 4):
        x[i] = 3
        x[i+1] = -1
        x[i+2] = 0
        x[i+3] = 1
    return x

nt(powell, x0_powell(4))

NT: STOP! The optimum point is: [ 0.00553509  0.01106667 -0.00110665  0.00553417] The optimum value of the objective function is: 3.1339119924221176e-08


array([ 0.00553509,  0.01106667, -0.00110665,  0.00553417])

### Dixon Function

In [10]:
def dixon(x):
    return (x[0]-1)**2 + sum(i*(2*(x[i-1]**2)-x[i-1-1])**2 for i in range(2,len(x)+1))

def x0_dixon(n):
    return np.ones(n)

nt(dixon, x0_dixon(4))

NT: STOP! The optimum point is: [1.00000001 0.70710679 0.59460356 0.54525387] The optimum value of the objective function is: 2.434268857130223e-16


array([1.00000001, 0.70710679, 0.59460356, 0.54525387])

### Box Function

In [11]:
def box(x):
    return sum((np.exp(-0.1*i*x[0])-np.exp(-0.1*i*x[1])-x[2]*(np.exp(-0.1*i)-np.exp(-i)))**2 for i in range(1, 11))

def x0_box():
    return np.array([0. ,10. ,20.])

nt(box, x0_box())

NT: STOP! The optimum point is: [ 0.99925515 10.01046526  1.00055092] The optimum value of the objective function is: 5.0312378567305434e-08


array([ 0.99925515, 10.01046526,  1.00055092])

### Oren Function

In [12]:
def oren(x):
    f = 0
    for i in range (0,len(x)):
        f += (i*(x[i]**2))
    return f**2

def x0_oren(n):
    x_1 = np.zeros(n)
    for i in range (0,n):
        x_1[i] = 1
    return x_1 

nt(oren, x0_oren(4))

NT: STOP! The optimum point is: [1.00000000e+00 1.35602946e-02 2.77855991e-04 1.04515931e-05] The optimum value of the objective function is: 3.386936940310604e-08


array([1.00000000e+00, 1.35602946e-02, 2.77855991e-04, 1.04515931e-05])

### Separated Rose Function

In [13]:
def separatedRose(x):
    f = 0
    c = 10.**1
    for j in range(0,len(x),2):
        f = f + (1. - x[j])**2 + c*(x[j+1] - x[j]**2)**2
    return f

def x0_separatedRose(n):
    x = np.zeros(n)
    for i in range(0,n,2):
        x[i] = -1.2
        x[i+1] = 1
    return x

nt(separatedRose, x0_separatedRose(4))

NT: STOP! The optimum point is: [0.99999401 0.99998768 0.99999401 0.99998768] The optimum value of the objective function is: 7.41272103708261e-11


array([0.99999401, 0.99998768, 0.99999401, 0.99998768])